In [24]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [25]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loaders = [
    # Duplicate documents on purpose - messy data
    PyPDFLoader("../docs/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("../docs/MachineLearning-Lecture01.pdf"),
    PyPDFLoader("../docs/MachineLearning-Lecture02.pdf"),
    PyPDFLoader("../docs/MachineLearning-Lecture03.pdf")
]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [26]:
docs

[Document(metadata={'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2008-07-11T11:25:23-07:00', 'author': '', 'moddate': '2008-07-11T11:25:23-07:00', 'title': '', 'source': '../docs/MachineLearning-Lecture01.pdf', 'total_pages': 22, 'page': 0, 'page_label': '1'}, page_content='MachineLearning-Lecture01  \nInstructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine \nlearning class. So what I wanna do today is just spend a little time going over the logistics \nof the class, and then we\'ll start to talk a bit about machine learning.  \nBy way of introduction, my name\'s Andrew Ng and I\'ll be instructor for this class. And so \nI personally work in machine learning, and I\'ve worked on it for about 15 years now, and \nI actually think that machine learning is the most exciting field of all the computer \nsciences. So I\'m actually always excited about teaching this class. Sometimes I actually \nthink that machine 

In [27]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [28]:
splits = text_splitter.split_documents(docs)

In [29]:
len(splits)

208

In [30]:
# embeddings

In [31]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [32]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [33]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [34]:
import numpy as np

In [35]:
# higher the np.dot value that means the sentence are relatively more syntactically similar

In [36]:
np.dot(embedding1, embedding2)

np.float64(0.9631510802407719)

In [37]:
np.dot(embedding1, embedding3)

np.float64(0.7702031204123156)

In [38]:
np.dot(embedding2, embedding3)

np.float64(0.7590539714454778)

In [44]:
# vector store

In [45]:
from langchain.vectorstores import Chroma

In [46]:
persist_directory = '../docs/chroma/'

In [47]:
rm -rf ./docs/chroma  # remove old database files if any

In [48]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [49]:
print(vectordb._collection.count())

208


In [50]:
question = "is there an email i can ask for help"

In [53]:
docs = vectordb.similarity_search(question,k=3)

In [54]:
len(docs)

3

In [55]:
docs[0].page_content

"cs229-qa@cs.stanford.edu. This goes to an account that's read by all the TAs and me. So \nrather than sending us email individually, if you send email to this account, it will \nactually let us get back to you maximally quickly with answers to your questions.  \nIf you're asking questions about homework problems, please say in the subject line which \nassignment and which question the email refers to, since that will also help us to route \nyour question to the appropriate TA or to me appropriately and get the response back to \nyou quickly.  \nLet's see. Skipping ahead — let's see — for homework, one midterm, one open and term \nproject. Notice on the honor code. So one thing that I think will help you to succeed and \ndo well in this class and even help you to enjoy this class more is if you form a study \ngroup.  \nSo start looking around where you're sitting now or at the end of class today, mingle a \nlittle bit and get to know your classmates. I strongly encourage you to form st

In [56]:
vectordb.persist()

/var/folders/q8/wt86p9kd3rd1ksjbgp7kvqt80000gn/T/ipykernel_7247/3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [57]:
"""
Failure modes
This seems great, and basic similarity search will get you 80% of the way there very easily.

But there are some failure modes that can creep up. Check README for same.

Notice that we're getting duplicate chunks (because of the duplicate MachineLearning-Lecture01.pdf in the index).

Semantic search fetches all similar documents, but does not enforce diversity.

docs[0] and docs[1] are indentical.
"""

"\nFailure modes\nThis seems great, and basic similarity search will get you 80% of the way there very easily.\n\nBut there are some failure modes that can creep up. Check README for same.\n\nNotice that we're getting duplicate chunks (because of the duplicate MachineLearning-Lecture01.pdf in the index).\n\nSemantic search fetches all similar documents, but does not enforce diversity.\n\ndocs[0] and docs[1] are indentical.\n"

In [58]:
question = "what did they say about matlab?"

In [59]:
docs = vectordb.similarity_search(question,k=5)

In [60]:
docs[0]

Document(metadata={'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2008-07-11T11:25:23-07:00', 'moddate': '2008-07-11T11:25:23-07:00', 'total_pages': 22, 'source': '../docs/MachineLearning-Lecture01.pdf', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'page': 8, 'page_label': '9', 'title': '', 'author': ''}, page_content='those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people call it a free version of MATLAB, which it sort of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t seen MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to \nwrite codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your own home computer or something if you \ndon\'t have a MATLAB

In [61]:
docs[1]

Document(metadata={'moddate': '2008-07-11T11:25:23-07:00', 'source': '../docs/MachineLearning-Lecture01.pdf', 'title': '', 'author': '', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2008-07-11T11:25:23-07:00', 'page_label': '9', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'page': 8, 'total_pages': 22}, page_content='those homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people call it a free version of MATLAB, which it sort of is, sort of isn\'t.  \nSo I guess for those of you that haven\'t seen MATLAB before, and I know most of you \nhave, MATLAB is I guess part of the programming language that makes it very easy to \nwrite codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it\'s sort of an extremely easy to learn tool to use for implementing a lot of \nlearning algorithms.  \nAnd in case some of you want to work on your own home computer or something if you \ndon\'t have a MATLAB

In [62]:
question = "what did they say about regression in the third lecture?"

In [63]:
docs = vectordb.similarity_search(question,k=5)

In [64]:
for doc in docs:
    print(doc.metadata)

{'source': '../docs/MachineLearning-Lecture03.pdf', 'moddate': '2008-07-11T11:25:03-07:00', 'total_pages': 16, 'page': 0, 'title': '', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'author': '', 'creationdate': '2008-07-11T11:25:03-07:00', 'creator': 'PScript5.dll Version 5.2.2', 'page_label': '1'}
{'creationdate': '2008-07-11T11:25:05-07:00', 'page': 2, 'total_pages': 18, 'title': '', 'creator': 'PScript5.dll Version 5.2.2', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'page_label': '3', 'source': '../docs/MachineLearning-Lecture02.pdf', 'moddate': '2008-07-11T11:25:05-07:00', 'author': ''}
{'total_pages': 18, 'creationdate': '2008-07-11T11:25:05-07:00', 'source': '../docs/MachineLearning-Lecture02.pdf', 'creator': 'PScript5.dll Version 5.2.2', 'title': '', 'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'author': '', 'page_label': '18', 'page': 17, 'moddate': '2008-07-11T11:25:05-07:00'}
{'title': '', 'total_pages': 22, 'page_label': '9', 'creationdate': '2008-07-11T11:25:23-

In [65]:
print(docs[4].page_content)

into his office and he said, "Oh, professor, professor, thank you so much for your 
machine learning class. I learned so much from it. There's this stuff that I learned in your 
class, and I now use every day. And it's helped me make lots of money, and here's a 
picture of my big house."  
So my friend was very excited. He said, "Wow. That's great. I'm glad to hear this 
machine learning stuff was actually useful. So what was it that you learned? Was it 
logistic regression? Was it the PCA? Was it the data networks? What was it that you 
learned that was so helpful?" And the student said, "Oh, it was the MATLAB."  
So for those of you that don't know MATLAB yet, I hope you do learn it. It's not hard, 
and we'll actually have a short MATLAB tutorial in one of the discussion sections for 
those of you that don't know it.  
Okay. The very last piece of logistical thing is the discussion sections. So discussion 
sections will be taught by the TAs, and attendance at discussion sections is o